Step 1: Importing dependencies.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
from scipy import stats
from google.colab import drive
import os

Step 2: Visualization config

In [ ]:
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

Step 3: defining variable and preparing csv file

In [ ]:
csv_path = "/content/synthetic_stock_data.csv"

Step 4: first visualization data

In [ ]:
# --- Verificar si el archivo existe en la ruta ---
if os.path.exists(csv_path):
    print(f"Archivo encontrado en: {csv_path}")
    # --- Cargar el conjunto de datos ---
    try:
        df = pd.read_csv(csv_path)
        print("¡DataFrame cargado exitosamente!")

        # ==================================================
        # 1.    Carga inicial del proyecto.
        # ==================================================
        print("\n----- 1. Carga e Inspección Inicial -----")

        # Ver las primeras y últimas filas
        print("\nPrimeras 5 filas:")
        print(df.head())
        print("\nÚltimas 5 filas:")
        print(df.tail())

        # Obtener dimensiones
        print(f"\nDimensiones del DataFrame (filas, columnas): {df.shape}")

        # Revisar tipos de datos y valores no nulos
        print("\nInformación del DataFrame (Tipos de datos y Nulos):")
        df.info()

        # Obtener estadísticas descriptivas básicas para variables numéricas
        print("\nEstadísticas Descriptivas (Variables Numéricas):")
        print(df.describe().T) # Usar .T para transponer y facilitar la lectura

        # Contar valores únicos por columna
        print("\nConteo de Valores Únicos por Columna:")
        print(df.nunique())

        # ... (Aquí pegarías el resto del código del EDA: Limpieza, Univariado, Bivariado, etc.) ...
        # El resto del código que te proporcioné anteriormente funcionará
        # a partir de aquí, ya que la variable 'df' ahora contiene tus datos.

    except Exception as e:
        print(f"Error al leer el archivo CSV: {e}")
        print("Asegúrate de que el archivo se subió correctamente y no está corrupto.")

else:
    print(f"¡ERROR! No se encontró el archivo en la ruta: {csv_path}")
    print("Por favor, asegúrate de haber subido el archivo 'synthetic_stock_data.csv' a tu sesión actual de Colab.")
    print("Puedes hacerlo usando el botón 'Subir archivo' en el panel de archivos de la izquierda.")

Step 5: handling duplicates

In [ ]:
# --- Manejo de Duplicados ---
print("\nNúmero de Filas Duplicadas:")
duplicates = df.duplicated().sum()
print(duplicates)

if duplicates > 0:
    print(f"\nSe encontraron {duplicates} filas duplicadas. Eliminándolas.")
    df.drop_duplicates(inplace=True)
    print(f"Nuevas dimensiones del DataFrame: {df.shape}")
else:
    print("\nNo se encontraron filas duplicadas.")

# --- Corrección de Tipos de Datos ---
print("\nVerificando y corrigiendo tipos de datos...")
# Convertir 'Date' a datetime si no lo está ya
if not pd.api.types.is_datetime64_any_dtype(df['Date']):
    try:
        df['Date'] = pd.to_datetime(df['Date'])
        print("'Date' convertida a tipo datetime.")
    except Exception as e:
        print(f"Error al convertir 'Date': {e}. Se mantendrá como objeto.")
else:
    print("'Date' ya es de tipo datetime.")

Step 6: Univariate analysis

In [ ]:
# ==================================================
# 2. Análisis Univariado
# ==================================================
print("\n----- 2. Análisis Univariado -----")

# Identificar columnas numéricas y categóricas (RE-IDENTIFICAR después de limpieza/corrección)
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df.select_dtypes(include='object').columns.tolist()
date_col = 'Date' # Definir explícitamente
# Asegurarse de no incluir columnas no deseadas si su tipo cambió
if 'Year' in numeric_cols: numeric_cols.remove('Year') # Si se crearon en una ejecución previa
if 'Month' in numeric_cols: numeric_cols.remove('Month')
if 'Day' in numeric_cols: numeric_cols.remove('Day')
if 'DayOfWeek' in numeric_cols: numeric_cols.remove('DayOfWeek')

print(f"\nColumnas Numéricas para Análisis Univariado: {numeric_cols}")
print(f"Columnas Categóricas para Análisis Univariado: {categorical_cols}")

# --- Variables Numéricas ---
print("\nAnálisis de Variables Numéricas:")

for col in numeric_cols:
    print(f"\n--- Análisis de '{col}' ---")
    if df[col].isnull().any():
        print(f"Advertencia: La columna '{col}' contiene valores NaN. Las estadísticas pueden verse afectadas.")
        # Considera df[col].dropna().skew() o .kurt() si quieres estadísticas sin NaNs
        skewness = "N/A (contiene NaN)"
        kurtosis = "N/A (contiene NaN)"
    else:
        skewness = f"{df[col].skew():.2f}"
        kurtosis = f"{df[col].kurt():.2f}"

    print(f"Media: {df[col].mean():.2f}")
    print(f"Mediana: {df[col].median():.2f}")
    print(f"Desviación Estándar: {df[col].std():.2f}")
    print(f"Mínimo: {df[col].min():.2f}")
    print(f"Máximo: {df[col].max():.2f}")
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    print(f"Q1: {Q1:.2f}")
    print(f"Q3: {Q3:.2f}")
    print(f"IQR (Rango Intercuartílico): {IQR:.2f}")
    print(f"Asimetría (Skewness): {skewness}")
    print(f"Curtosis: {kurtosis}")

    # Visualización: Histograma y Boxplot
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    # Usar dropna() para evitar errores en el plot si hay NaNs
    sns.histplot(df[col].dropna(), kde=True)
    plt.title(f'Distribución de {col}')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')

    plt.subplot(1, 2, 2)
    sns.boxplot(y=df[col].dropna()) # Usar dropna() aquí también
    plt.title(f'Boxplot de {col}')
    plt.ylabel(col)

    plt.tight_layout()
    plt.show()

# --- Variables Categóricas ---
print("\nAnálisis de Variables Categóricas:")

for col in categorical_cols:
    print(f"\n--- Análisis de '{col}' ---")

    # Frecuencias y Proporciones
    print("Conteo de Valores:")
    value_counts = df[col].value_counts()
    print(value_counts)
    print("\nProporción de Valores (%):")
    print(value_counts.div(value_counts.sum()).mul(100).round(2).astype(str) + '%')

    # Visualización: Gráfico de Barras (si no hay demasiadas categorías)
    num_unique = df[col].nunique()
    print(f"Número de categorías únicas: {num_unique}")
    if num_unique > 0 and num_unique < 30: # Umbral arbitrario y evitar error si columna está vacía
        plt.figure(figsize=(10, max(5, num_unique * 0.3))) # Ajustar altura dinámicamente
        sns.countplot(y=df[col], order = value_counts.index, palette='viridis')
        plt.title(f'Frecuencia de Categorías en {col}')
        plt.xlabel('Conteo')
        plt.ylabel(col)
        plt.tight_layout()
        plt.show()
    elif num_unique >= 30:
        print(f"No se genera gráfico de barras para '{col}' debido a la alta cardinalidad ({num_unique} categorías).")
        print("Top 10 categorías:")
        print(value_counts.head(10))
    else:
         print(f"La columna '{col}' no tiene valores o categorías únicas para graficar.")